In [ ]:
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
import random
from os.path import exists

import torch
import torch.nn as nn
import torch.optim as optim

from datetime import datetime, timedelta
from sklearn.decomposition import PCA

In [ ]:
def DataClean(f1,f2):
	with open(f1) as f:
		data = json.load(f)
	for i in data.values():
		for j in i:
			del j["place_vector"]
			del j["city_vector"]
	json.dump(data,open(f2,'w'),ensure_ascii=False)

In [ ]:
for i in range(3):
	if not exists('data_'+str(i)+'.json'):
		DataClean('classified_data_'+str(i)+'.json','data_'+str(i)+'.json')

In [ ]:
# with open('data_0.json') as f:
# 	d0 = json.load(f)
# with open('data_1.json') as f:
# 	d1 = json.load(f)
# with open('data_2.json') as f:
# 	d2 = json.load(f)

In [ ]:
def get_data(opt: int):
	with open('data_'+str(opt)+'.json') as f:
		data = json.load(f)
	return data
data = get_data(0)

In [ ]:
input_size = 0
for city in data.values():
	if input_size != 0:
		break
	for p in city:
		if type(p['patient_vector']) == list:
			input_size = len(p['patient_vector'])
			break

In [ ]:
input_size

In [ ]:
total = 0
useful = 0
train_city = set()
test_city = set()
for city in data.values():
	total += len(city)
	if len(city) > 60:
		test_city.add(city[0]['true_city'])
		train_city.add(city[0]['true_city'])
	elif len(city) > 30:
		train_city.add(city[0]['true_city'])
	else:
		continue
	for p in city:
		if type(p['patient_vector']) == list:
			useful += 1
print(train_city)
print(test_city)
print(total)
print(useful)

In [ ]:
# tset = set()
# for p in data['重庆']:
# 	tset.add(p['time'])
# tset

In [ ]:
temp = np.empty((useful,input_size))
temp_i = 0
for city_string in train_city:
	city = data[city_string]
	for p in city:
		if type(p['patient_vector']) == list:
			temp[temp_i] = p['patient_vector']
			temp_i += 1
##print(temp[:,:2])
print(temp.shape)
pca = PCA(n_components=5)
result = pca.fit_transform(temp)
print(result.shape)
print(result)

In [ ]:
temp_i = 0
for city_string in train_city:
	city = data[city_string]
	for p in city:
		if type(p['patient_vector']) == list:
			p['patient_vector'] = temp[temp_i]
			temp_i += 1

In [ ]:
# start_date = datetime.strptime('2022:03:08','%Y:%m:%d')
# end_date = datetime.strptime('2022:03:23','%Y:%m:%d')
# type((end_date-start_date).days)
# input_size

In [ ]:
input_size = result.shape[1]
datas = {}
for city_string in train_city:
	city = data[city_string]
	start_date = datetime.strptime(city[0]['time'],'%Y:%m:%d')
	end_date = datetime.strptime(city[-1]['time'],'%Y:%m:%d')
	# start_date = datetime.strptime('2022:03:08','%Y:%m:%d')
	# end_date = datetime.strptime('2022:03:23','%Y:%m:%d')
	# CNT = []
	# VECTOR = []
	# while start_date < end_date:
	# 	start_date += timedelta(days=1)
	# 	cnt = 0
	# 	vector = []
	# 	for p in city:
	# 		if p['time']==(start_date).strftime('%Y:%m:%d'):
	# 			cnt+=1
	# 			vector.append(p['patient_vector'])

	# 	if any((type(v)==list for v in vector)):
	# 		vector = np.array([v for v in vector if type(v)==list])
	# 		vector = np.mean(vector,axis=0)
	# 	else:
	# 		vector = np.zeros((input_size,))
	# 	CNT.append(cnt)
	# 	VECTOR.append(vector)
	# CNT = np.array(CNT)
	# VECTOR = np.array(VECTOR)
	# temp = np.hstack((CNT.reshape(-1,1),VECTOR))
	period = (end_date-start_date).days+1
	CNT = np.zeros((period))
	VECTOR = np.zeros((period, input_size))
	cnt = 0
	vector = []
	day_idx = 0
	for p in city:
		if p['time'] != (start_date).strftime('%Y:%m:%d'):
			if any((type(v)==list for v in vector)):
				vector = np.array([v for v in vector if type(v)==list])
				VECTOR[day_idx] = np.mean(vector,axis=0)
			CNT[day_idx] = cnt
			cnt = 0
			vector = []
		while p['time'] != (start_date).strftime('%Y:%m:%d'):
			start_date += timedelta(days=1)
			day_idx += 1
		cnt += 1
		vector.append(p['patient_vector'])
	else:
		if any((type(v)==list for v in vector)):
			vector = np.array([v for v in vector if type(v)==list])
			VECTOR[day_idx] = np.mean(vector,axis=0)
		CNT[day_idx] = cnt
	temp = np.hstack((CNT.reshape(-1,1),VECTOR))
	# if any(CNT[i-3:i,0]) or any(CNT[i+1:i,0]) 
	datas[city_string] = torch.tensor(temp.reshape(-1,1,input_size+1)).float()
# input_size += 1

In [ ]:
datas.keys()
# datas['重庆'].size()

In [ ]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
test_size = 1
len(datas)

In [ ]:
class Net(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(Net, self).__init__()
        self.gru = nn.GRU(input_size=input_size+1, \
            hidden_size=hidden_size,num_layers=2)
        self.linear = nn.Linear(hidden_size,output_size) 
        self.relu = nn.ReLU()

    def forward(self, x):
        output, h_n = self.gru(x)
        # output = self.relu(output[-1])
        output = self.linear(output)
        output = self.relu(output[-1])
        return output

net = Net(input_size,14,1)
# net.to(device)

In [ ]:

# create your optimizer
optimizer = optim.Adam(net.parameters(), lr=0.01)
criterion = nn.MSELoss()


In [ ]:
def data_iter(data,seq_len,test_size,in_test):
	if in_test:
		indexs = np.arange(len(data)-seq_len-test_size)
	else:
		indexs = np.arange(len(data)-seq_len)
	random.shuffle(indexs)
	for i in indexs:
		# print(data[i:i+7].shape)
		yield data[i:i+seq_len],data[i+seq_len][0][0]

In [ ]:
# city_keys = list(datas.keys())
# random.shuffle(city_keys)
# city_keys

In [ ]:
epoches = 200
seq_len = 7
for epoch in tqdm(range(epoches)):  # loop over the dataset multiple times

    running_loss = 0.0
    city_keys = list(datas.keys())
    random.shuffle(city_keys)
    for city in city_keys:
        
        for data in data_iter(datas[city],seq_len,test_size,city in test_city):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            # inputs, labels = inputs.to(device), labels.view(1,1).to(device)
            inputs, labels = inputs, labels.view(1,1)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

    if epoch%5 == 4:
        print(epoch+1,running_loss)

    running_loss = 0.0

print('Finished Training')

In [ ]:
for i in range(test_size):
	for city_string in test_city:
		inputs = datas[city_string][-i-seq_len-1:-i-1]
		outputs = net(inputs)
		print(city_string,int(outputs)+1, \
			int(datas[city_string][-i-1][0][0]))

In [ ]:
# for city_string in train_city:

# 	inputs = datas[city_string][-9:-2]
# 	outputs = net(inputs)
# 	print(city_string,int(outputs)+1,int(datas[city_string][-2][0][0]))

In [ ]:
# for i in net.gru.parameters():
# 	print(i)